DocMate is AI chat assistant that answers your medical questions
,lets login to hungging face with access token

In [1]:
!pip install huggingface_hub

from huggingface_hub import HfFolder, whoami

hf_token = 'YOUR_TOKEN'
HfFolder.save_token(hf_token)

user = whoami()
print(f"Logged in as {user['name']}")


Logged in as Akhil1905


In the below cell we consider a model from hugging face which will be by default FP32 precision 
Our goal is to compress this model with OpenVino 
Precisions considered are FP16, INT8, INT4

Gemma 2b-It: Gemma is a family of lightweight, state-of-the-art open models from Google, built from the same research and technology used to create the Gemini models. They are text-to-text, decoder-only large language models, available in English, with open weights, pre-trained variants, and instruction-tuned variants. Gemma models are well-suited for a variety of text generation tasks, including question answering, summarization, and reasoning. Their relatively small size makes it possible to deploy them in environments with limited resources such as a laptop, desktop or your own cloud infrastructure, democratizing access to state of the art AI models and helping foster innovation for everyone.

In [3]:
!pip install tf-keras==2.16.0
!pip install nncf
!pip install optimum.intel
!pip install openvino
model_id = 'google/gemma-2b-it'
model_vendor, model_name = model_id.split('/')

FP16_GEN, INT8_GEN, INT4_GEN = True, True, True

if FP16_GEN:
    from optimum.intel.openvino import OVModelForCausalLM
    ov_model_int8=OVModelForCausalLM.from_pretrained(model_id=model_id, export=True, compile=False, load_in_8bit=False)
    ov_model_int8.half()
    ov_model_int8.save_pretrained(f'{model_name}/FP16')

if INT8_GEN:
    from optimum.intel.openvino import OVModelForCausalLM
    from optimum.intel import OVQuantizer
    ov_model_int8=OVModelForCausalLM.from_pretrained(model_id=model_id, export=True, compile=False, load_in_8bit=False)
    quantizer = OVQuantizer.from_pretrained(ov_model_int8)
    quantizer.quantize(save_directory=f'{model_name}/INT8', weights_only=True)

if INT4_GEN:
    import shutil
    from optimum.intel.openvino import OVModelForCausalLM
    import openvino as ov
    import nncf
    ov_model_int8=OVModelForCausalLM.from_pretrained(model_id=model_id, export=True, compile=False, load_in_8bit=False)
    compressed_model = nncf.compress_weights(ov_model_int8.half()._original_model, mode=nncf.CompressWeightsMode.INT4_ASYM, group_size=128, ratio=0.8)
    ov.save_model(compressed_model, f'{model_name}/INT4/openvino_model.xml')
    shutil.copy(f'{model_name}/FP16/config.json', f'{model_name}/INT4/config.json')

/opt/app-root/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
2024-11-15 19:00:54.242406: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 19:00:54.253733: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 19:00:54.269906: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 19:00:54.26994

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)
/opt/app-root/lib64/python3.9/site-packages/transformers/cache_utils.py:458: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  or len(self.key_cache[layer_idx]) == 0  # the layer has no cache
/opt/app-root/lib64/python3.9/site-packages/optimum/exporters/openvino/model_patcher.py:496: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a cons

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/app-root/lib64/python3.9/site-packages/transformers/cache_utils.py:458: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  or len(self.key_cache[layer_idx]) == 0  # the layer has no cache
/opt/app-root/lib64/python3.9/site-packages/optimum/exporters/openvino/model_patcher.py:496: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if sequence_length != 1:
/opt/app-root/lib64/python3.9/site-packages/nncf/torch/dynamic_graph/wrappers.py:86: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of co

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/app-root/lib64/python3.9/site-packages/transformers/cache_utils.py:458: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  or len(self.key_cache[layer_idx]) == 0  # the layer has no cache
/opt/app-root/lib64/python3.9/site-packages/optimum/exporters/openvino/model_patcher.py:496: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if sequence_length != 1:
/opt/app-root/lib64/python3.9/site-packages/nncf/torch/dynamic_graph/wrappers.py:86: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of co

Output()

INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│   Num bits (N) │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│              8 │ 38% (30 / 127)              │ 21% (29 / 126)                         │
├────────────────┼─────────────────────────────┼────────────────────────────────────────┤
│              4 │ 62% (97 / 127)              │ 79% (97 / 126)                         │
┕━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙


Output()

Now a folder will be created after this step with compressed models in the form of OpenVino IR
We can load these models by considering their respective directories.

I am considering mainly FP16 and INT 8 models would like to use them interchangebaly depending on the complexity of task

In [4]:

from transformers import AutoConfig, AutoTokenizer
from optimum.intel.openvino import OVModelForCausalLM

import openvino as ov
import openvino.properties as props
import openvino.properties.hint as hints
import openvino.properties.streams as streams

core = ov.Core()
model_vendor, model_name = model_id.split('/')

model_precision = ['FP16', 'INT8', 'INT4', 'INT4_stateless'][1]

model_dir=f'{model_name}/{model_precision}'
tok = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
ov_config = {hints.performance_mode(): hints.PerformanceMode.LATENCY, streams.num(): "1", props.cache_dir(): ""}
ov_model_int8 = OVModelForCausalLM.from_pretrained(
    model_dir,
    device='CPU',
    ov_config=ov_config,
    config=AutoConfig.from_pretrained(model_dir, trust_remote_code=True),
    trust_remote_code=True,
)
model_precision = ['FP16', 'INT8', 'INT4', 'INT4_stateless'][0]
ov_model_fp16=OVModelForCausalLM.from_pretrained(
    model_dir,
    device='CPU',
    ov_config=ov_config,
    config=AutoConfig.from_pretrained(model_dir, trust_remote_code=True),
    trust_remote_code=True,
)
print('Loaded 2 models')


Loaded 2 models


In [5]:
# Lets test the 2 models with same input
prompt_text = 'Explain about digestive system.\n\n'
input_tokens = tok.encode(prompt_text, return_tensors='pt')
response = ov_model_int8.generate(input_tokens, max_new_tokens=300) 
response_text = tok.decode(response[0], skip_special_tokens=True)
response_1=ov_model_fp16.generate(input_tokens, max_new_tokens=300)   
response_text_1 = tok.decode(response_1[0], skip_special_tokens=True)
print(f'Response from 1st model INT 8:{response_text}')
print(f'Response from 2nd model FP 16:{response_text_1}')

Response from 1st model INT 8:Explain about digestive system.

Sure, here's a detailed explanation of the digestive system:

**The digestive system** is a complex network of organs and tissues that work together to break down food, absorb nutrients, and eliminate waste products. It can be divided into two main parts: the **mechanical digestive system** and the **chemical digestive system**.

**The mechanical digestive system** physically breaks down food into smaller molecules that can be absorbed by the body. This system includes the mouth, esophagus, stomach, and intestines.

* **The mouth** is the first part of the digestive system. It breaks down food into smaller pieces by chewing and saliva.
* **The esophagus** is a tube that carries food from the mouth to the stomach.
* **The stomach** is a J-shaped organ that secretes gastric juices, which help to break down proteins.
* **The intestines** are a series of long, coiled tubes that absorb nutrients from food that has been digested 

Here we can observe both the reponses are pretty similar because we asked a pretty simple quetion. The outputs will be visibly differ if we give any complex question

The below code is used for implementing retieval through PubMed Search
PubMed is a free, publicly available search engine for biomedical and life sciences literature: 
What it contains
PubMed contains over 37 million citations and abstracts from journals in medicine, nursing, dentistry, veterinary medicine, and other related fields. It also includes links to full text articles when available. 
Who developed it
PubMed was developed and is maintained by the National Center for Biotechnology Information (NCBI) at the National Library of Medicine (NLM) at the National Institutes of Health (NIH)

In [6]:
!pip install biopython
from Bio import Entrez
from bs4 import BeautifulSoup

def pubmed_search(query):
    """
    Search PubMed for articles matching the query and return the articles' titles and abstracts.
    
    Params:
    query (str): The search query to look for articles in PubMed.
    
    Returns:
    list: A list of tuples containing article titles and abstracts.
    """
    # Set your email (required by NCBI API)
    Entrez.email = "YOUR_EMAIL"

    # Perform the PubMed search
    handle = Entrez.esearch(db="pubmed", term=query, retmax=3)  # Limit to 3 articles for simplicity
    record = Entrez.read(handle)
    handle.close()

    # Get the list of PubMed article IDs (PMID)
    pmids = record["IdList"]
    articles = []

    # Fetch details for each article using the IDs
    if pmids:
        handle = Entrez.efetch(db="pubmed", id=pmids, rettype="xml", retmode="text")
        records = Entrez.read(handle)
        handle.close()

        # Process each record
        for article in records["PubmedArticle"]:
            title = article["MedlineCitation"]["Article"]["ArticleTitle"]
            try:
                abstract = article["MedlineCitation"]["Article"]["Abstract"]["AbstractText"][0]
            except KeyError:
                abstract = "No abstract available."

            # Truncate abstract to a maximum length of 400 words
            truncated_abstract = truncate(abstract)

            # Append the article title and truncated abstract to the result list
            articles.append((title, truncated_abstract))

    return articles


def truncate(text):
    """
    Truncate text to 400 words.

    Params:
    text (str): The text to be truncated.

    Returns:
    str: The truncated text.
    """
    words = text.split()
    return " ".join(words[:400])


# Example usage:
query = "cancer immunotherapy"
articles = pubmed_search(query)

for title, abstract in articles:
    print(f"Title: {title}")
    print(f"Abstract: {abstract}")
    print("\n---\n")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 43.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Title: Cowpea mosaic virus intratumoral immunotherapy maintains stability and efficacy after long-term storage.
Abstract: Cowpea mosaic virus (CPMV) has demonstrated superior immune stimulation and efficacy as an intratumoral immunotherapy, providing a strong argument for its clinical translation. One important consideration for any new drug candidate is the long-term stability of the drug and its formulation. Therefore, our lab has evaluated the physical stability and biological activity, that is, anti-tumor potency, of formulations of CPMV in buffer (with and without a sucrose preservative) in multiple temperature conditions ranging from ultralow freezers to a heated incubator over a period of 9 months. We found that non-refrigerated temperatures 37°C and room temperature quickly led to CP

The above are few articles printed for a search through PubMed:
The format it provides contains a title and an abstract about that

Below I have Implemented the Core logic for the chat application :

The main idea is to intent based classification of user input

For General Purpose which is basically handling genral medical questions I have considered INT8 model Which can handle efficiently

For more sophisticated responses like summarizing or answering from PubMed Search or Disesasem prediction I have used FP16 

The usage of this Hybrid approach makes the application a bit more efficient in handling user queries

In [28]:
# !pip install gradio
# !pip install langchain_community
import gradio as gr
import time
from sklearn.preprocessing import LabelEncoder
import textwrap
import pandas as pd
import tensorflow as tf
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer



def process_response(text):
    """Helper function to process Gemini responses"""
    input_tokens = tok.encode(text, return_tensors='pt')
    response = ov_model_int8.generate(input_tokens, max_new_tokens=300,temperature=0.3)   # Other options: temperature=1.0, do_sample=True, top_k=5, top_p=0.85, repetition_penalty=1.2)
    response_text = tok.decode(response[0], skip_special_tokens=True)
    return response_text.replace(text,'')

def get_disease(input_text):
    # Extract symptoms
    
    prompt = f"""
You are a medical chat assistant. The user has reported symptoms, and based on this information, you should:
1. Predict the most likely disease.
2. Highlight any additional symptoms that might be related to the disease.
3. Provide the top 5 remedies with a brief explanation for each remedy.
Please ensure that the response is clear, concise, and appropriate for a non-technical audience.
User reported symptoms: {input_text}
Note: You should not generate user specific responses or prompt in th final response
"""
   
    input_tokens = tok.encode(prompt, return_tensors='pt')
    response = ov_model_fp16.generate(input_tokens, max_new_tokens=300)   # Other options: temperature=1.0, do_sample=True, top_k=5, top_p=0.85, repetition_penalty=1.2)
    response_text = tok.decode(response[0], skip_special_tokens=True)
   
    response_text.replace('User Input:','')
    response_text.replace(input_text,'')
    
    return response_text.replace(prompt,'')
def get_scimed(input_text):
    articles = pubmed_search(input_text)
    articles_text = "\n".join(articles)
    
    prompt = f"""
    You are a medical chat assistant.
    Based on the articles below, provide information for the user’s query in a friendly and professional manner.
    user' query:{input_text}
    Articles:
    {articles_text}
    
    """
    input_tokens = tok.encode(prompt, return_tensors='pt')
    
    response = ov_model_fp16.generate(input_tokens, max_new_tokens=300,do_sample=True)   # Other options: temperature=1.0, do_sample=True, top_k=5, top_p=0.85, repetition_penalty=1.2)
    response_text = tok.decode(response[0], skip_special_tokens=True)
    print(response_text)
    response_text.replace('User Input:','')
    response_text.replace(input_text,'')
    ls=[]
    
    for title in articles:
        ls.append(title)
        
    return response_text.replace(prompt,''),ls


def chat_response(message, history):
    # Classify the user input based on intent
    intent_prompt = f'''Please classify the following input into one of the categories below. Analyze the context and nuances of the input carefully to ensure accurate classification.

**Categories:**
1. **New Symptom**: Use this category if the user describes symptoms they are currently experiencing, without asking a question about a symptom, disease, or medical condition in general. This is typically when the user is directly seeking help for their own health concerns (e.g., "I have a sore throat and feel dizzy" or "I'm experiencing joint pain").

2. **Clarification**: Use this category if the user asks for specific clarification on a medical topic, symptom, or term they don’t fully understand. This usually involves requests for explanation, interpretation, or more information about something specific (e.g., "What does hypertension mean?" or "Can you clarify the side effects of this drug?").

3. **General Info Request**: Use this category if the user asks general medical questions, seeks information on diseases, or makes inquiries that do not involve their own symptoms. This includes questions about conditions, lifestyle advice, and preventive health measures (e.g., "What causes diabetes?" or "How can I improve my immune system?").

4. **Scientific Doubt**: Use this category for inquiries related to recent or advanced scientific research, experimental treatments, new technologies, or cutting-edge findings in the medical field. This typically involves complex or specialized questions (e.g., "What are the latest developments in cancer immunotherapy?" or "How does CRISPR technology work in treating genetic diseases?").

5. **Other**: Use this category for all other types of inquiries that do not fit the above categories. This may include conversational or off-topic messages, general greetings, or questions outside the medical domain (e.g., "Hello, how are you?" or "Can you help me with a non-medical question?").

**Instructions**:
- Focus on the context and intent behind the input to classify it correctly.
- Consider subtle differences; for example, distinguish between asking for information about a symptom (General Info Request) and describing one’s own symptoms (New Symptom).
- Only provide the class name.

Input: {message}'''


    # Get the intent by processing the classification prompt
    intent = process_response(intent_prompt)
    intent = intent.strip()  # Clean up the output
    print(intent)
    
    # Now generate a response based on the classified intent
    if 'New Symptom' in intent:
        # Generate a response and update the history
        response = get_disease(message)  
        history.append(("user", message))
        history.append(("bot", response))
        
    elif 'Clarification' in intent:
        # Provide a clarification response while keeping the tone friendly
        prompt = f'You are a doctor. Clarify this request: {message}. Sound friendly and professional.'
        response = process_response(prompt)
        history.append(("user", message))
        history.append(("bot", response))
        
    elif 'General Info Request' in intent:
        # Provide detailed medical information in an easy-to-understand format
        prompt = f'You are a doctor. Provide detailed yet easy-to-understand medical info for: {message}'
        response = process_response(prompt)
        history.append(("user", message))
        history.append(("bot", response))
        
    elif 'Scientific Doubt' in intent:
        # Handle scientific-related medical inquiries
        prompt = f'You are a doctor. Provide scientific details about: {message}.'
        response,references = get_scimed(prompt)
        response, references = get_scimed(prompt)
        if references:
            response += "\n\nReferences:\n"
            for reference in references:
                response += f"- {reference}\n"
        history.append(("user", message))
        history.append(("bot", response))

    else:  # Other
        # Placeholder for better output 
        response = ' I am a medical Assistant designed specifically for medical use cases. I may not be good one to answer the specific question. Kindly provide any medical related queries'
        history.append(("user", message))
        history.append(("bot", response))

    # Update history with the latest response
    return response



Each of these functions with specific prompts helps in efficient and a bit accurate responsed to the user query

In [29]:
# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("""# DocMate a Chat Assistant
    👋 Welcome to the DocMate Chat Assistant.  
    Your personal medical chat assistant, connected to the internet world.🌎
    """)
    
    chatbot = gr.ChatInterface(
        fn=chat_response,
        examples=[
            "I have fever, headache, and body pain",
            "Can you explain what causes high blood pressure?",
            "Latest cancer treatments?"
        ],
        title="Powered by OpenVino with Gemma 2B",
        retry_btn=None,
        undo_btn=None,
        clear_btn="Clear"
    )

if __name__ == "__main__":
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7868


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running on public URL: https://9d8816331f52b5cea8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/opt/app-root/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


**Answer:** 4. Scientific Doubt

The question is about the latest developments in cancer immunotherapy, which is a scientific research area.

    You are a medical chat assistant.
    Based on the articles below, provide information for the user’s query in a friendly and professional manner.
    user' query:You are a doctor. Provide scientific details about: Latest cancer treatments?.
    Articles:
    
    
    1. **Recent Clinical Trials on Cancer Treatment** published in "The Lancet" reports promising results of a new targeted therapy for leukemia. The therapy, called Pembrolizumab (MK-452), significantly improved the survival rate of patients with aggressive leukemia and was found to be safe and well-tolerated. 

    2. **New Drug Approvals for Cancer Treatment** published in "The New England Journal of Medicine" announced the approval of a novel immunotherapy drug called Nivolumab (Opdivo) for treating multiple myeloma. Nivolumab is a chimeric antigen receptor (CAR) drug targeting

The chat application can be interacted via gradio interface and the public URL generated.

The application considers the users symptoms and predicts most likely disease from it. It also provides additional symptoms that user can counter verify if they have these additional symptoms or not. Finally our application provides remedies for temporary relief for the user's symptoms. The other thing it is capable of is it can handle queries that requires latest medical treatments.